In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv("../crawling/merged_csv.csv")

In [3]:
print(df.shape)
df.head()

(11101, 5)


,Unnamed: 0,Unnamed: 0.1,category,header,content
0,0,0,politics,Mary Trump slams Susan Sarandon as a 'moron' f...,Mary Trump criticized longtime activist and ac...
1,1,1,politics,Moment two Ukrainian fighter jets bomb Russian...,New footage shows the moment that Ukrainian A...
2,2,2,politics,Mark Esper on what's at stake if Donald Trump ...,Former Secretary of Defense Mark Esper was con...
3,3,3,politics,Hunter Biden enlists Hollywood mega-lawyer for...,Hunter Biden has garnered quiet support and fi...
4,4,4,politics,President Trump's response to Mark Esper's 60 ...,When asked for comment on this week's intervie...


In [4]:
df.drop(columns="Unnamed: 0", axis=1, inplace=True)
df.drop(columns="Unnamed: 0.1", axis=1, inplace=True)

In [5]:
df

,category,header,content
0,politics,Mary Trump slams Susan Sarandon as a 'moron' f...,Mary Trump criticized longtime activist and ac...
1,politics,Moment two Ukrainian fighter jets bomb Russian...,New footage shows the moment that Ukrainian A...
2,politics,Mark Esper on what's at stake if Donald Trump ...,Former Secretary of Defense Mark Esper was con...
3,politics,Hunter Biden enlists Hollywood mega-lawyer for...,Hunter Biden has garnered quiet support and fi...
4,politics,President Trump's response to Mark Esper's 60 ...,When asked for comment on this week's intervie...
...,...,...,...
11096,food,Classic Tater Tot Casserole with Green Beans,This is one of my favorite childhood recipe's....
11097,food,Stews of South India,Know about different stews present in South In...
11098,food,Biscuits from Self Rising Flour,Biscuits are a learning experience. Best buy e...
11099,food,Streusel Topping for Banana Bread for Example,Banana bread topping is added just before you ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11101 entries, 0 to 11100
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  11101 non-null  object
 1   header    11101 non-null  object
 2   content   9499 non-null   object
dtypes: object(3)
memory usage: 260.3+ KB


In [7]:
df[df['content'].isna()]

,category,header,content
211,technology,Technology to Make Your Life Easier,NaN
214,technology,Steps to Automating Your Production Plant,NaN
240,technology,How to Cancel a Subscription on iPhone,NaN
243,technology,What Tools Are Needed for 5G mmWave Testing?,NaN
246,technology,TRXmining : The Next Decentralized Finance Hub,NaN
...,...,...,...
11052,food,Kung Pao Chicken Introduction,NaN
11053,food,One of the Most Popular Snack and Dessert Tofu...,NaN
11054,food,Traditional Chinese Snack -- Stinky Tofu,NaN
11093,food,No Flour Chocolate Cake,NaN


In [8]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9499 entries, 0 to 11100
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  9499 non-null   object
 1   header    9499 non-null   object
 2   content   9499 non-null   object
dtypes: object(3)
memory usage: 296.8+ KB


In [9]:
#pd.set_option('display.max_rows', None)
df[df.duplicated() == True]

,category,header,content
753,technology,Motivational Sayings Great Ones,Why examine motivational sayings? For motivati...
754,technology,5 Reasons Why the Fire TV Stick 4k Is the Top ...,The Fire TV Stick 4K has quickly become the to...
755,technology,The Anker PowerConf H700 AI-Powered Wireless H...,A headset that mitigates noise and lets you ta...
756,technology,Four Gadgets that will Enhance Your Cycling Ex...,"Friends, through this article of today, we are..."
757,technology,Headphones On and World Off With Happy Plugs A...,The Happy Plugs Air 1 ANC True Wireless Earbud...
...,...,...,...
11088,food,"When Life Gives You Lemons, Reap Their Health ...",From preventing heart disease and cancer to im...
11089,food,Poi Besar (Malabar spinach mustard curry / Bas...,"The end of spring and beginning of summer, the..."
11090,food,Matar Paneer - Indian Cuisine,Matar Paneer - In Hindi Matar means Green peas...
11091,food,What Is Structured Water and How to Prepare it...,"Learn what structured water is, how to make it..."


In [10]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6710 entries, 0 to 11100
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6710 non-null   object
 1   header    6710 non-null   object
 2   content   6710 non-null   object
dtypes: object(3)
memory usage: 209.7+ KB


In [11]:
# Remove numbers
for i, row in df.iterrows():    
    row["content"] = re.sub(r'\d','', row["content"])
    row["header"] = re.sub(r'\d','', row["header"])
    
df

,category,header,content
0,politics,Mary Trump slams Susan Sarandon as a 'moron' f...,Mary Trump criticized longtime activist and ac...
1,politics,Moment two Ukrainian fighter jets bomb Russian...,New footage shows the moment that Ukrainian A...
2,politics,Mark Esper on what's at stake if Donald Trump ...,Former Secretary of Defense Mark Esper was con...
3,politics,Hunter Biden enlists Hollywood mega-lawyer for...,Hunter Biden has garnered quiet support and fi...
4,politics,President Trump's response to Mark Esper's Mi...,When asked for comment on this week's intervie...
...,...,...,...
11096,food,Classic Tater Tot Casserole with Green Beans,This is one of my favorite childhood recipe's....
11097,food,Stews of South India,Know about different stews present in South In...
11098,food,Biscuits from Self Rising Flour,Biscuits are a learning experience. Best buy e...
11099,food,Streusel Topping for Banana Bread for Example,Banana bread topping is added just before you ...


In [12]:
from langdetect import detect

wrong_languages = ["ko", "hi", "ml", "ru"]

for i, row in df.iterrows():
    try:           
        content_language = detect(row["content"])
        if content_language in wrong_languages:
            df.drop(i, inplace=True)
            print()
            print(f"!!! Detect content as {content_language} language.\nDrop row form df:\n{row}")
            print()
            
            df.drop(i, inplace=True)        
            
    except Exception as error:
        print(f"Drop row form df:\n {row}")
        df.drop(i, inplace=True)    

(6710, 3)
Detect content as no language.
Drop row form df:
category                                   technology
header      Emails - Deleting from Gmail - One Method
content            I have to start deleting emails...
Name: 376, dtype: object

-----------------------------

Detect content as sv language.
Drop row form df:
category                                           technology
header      Photos - Transferring from DVD to Hard Drive -...
content                          Installing photos from disk.
Name: 682, dtype: object

-----------------------------

Detect content as pt language.
Drop row form df:
category                                              science
header                                           Quem sou eu?
content     Além da aspirante a poeta que já publicou dive...
Name: 1322, dtype: object

-----------------------------

Detect content as ca language.
Drop row form df:
category                       science
header      Physics: The Heat Capacity
content   

In [13]:
import re

pd.set_option('display.max_rows', None)
for i, row in df.iterrows():        
    row["content"] = row["content"].replace("\n"," ")
    row["content"] = row["content"].replace("\r"," ")
    row["content"] = re.sub(r'[^a-zA-Z\s]','', row["content"])

    row["header"] = row["header"].replace("\n"," ")
    row["header"] = row["header"].replace("\r"," ")        
    row["header"] = re.sub(r'[^a-zA-Z\s]','', row["header"])
    

In [14]:
#TODO: What to do?
df = df[df["content"].apply(len) > 20]